In [ ]:
from ipyleaflet import (
    Map,
    Marker,
    Popup,
    LayerGroup,
    MarkerCluster,
    WidgetControl,
    AwesomeIcon,
)
from ipywidgets import HTML, IntSlider, jslink, Button, Layout
import db

In [ ]:
def reset(self):
    m.center = (38, -90)
    m.zoom = 4


m = Map(height="800px")
reset(None)

map_markers = LayerGroup()
m.add(map_markers)

button = Button(icon="home", tooltip="Reset Zoom", layout=Layout(width="35px"))
button.on_click(reset)
m.add(WidgetControl(widget=button, position="topleft"))

max_distance_slider = IntSlider(
    value=400,
    min=20,
    max=1600,
    step=20,
    description="Max Distance:",
    style=dict(description_width="100px"),
    continuous_update=False,
)
m.add(WidgetControl(widget=max_distance_slider, position="topright"))

display(m)

In [ ]:
def draw_markers(change):
    map_markers.clear_layers()

    rows = db.query(
        f"""
    select code, station_lat, station_lon, bikeshare_lat, bikeshare_lon, station_name, bikeshare_name, dist from int_amtrak_stations_to_bikeshare 
    where dist < {int(change['new'])} order by dist asc
    """
    ).fetchall()

    stations = {}

    for row in rows:
        if row[0] not in stations:
            stations[row[0]] = []
        stations[row[0]].append(row[1:])

    for code, rows in stations.items():
        markers = []

        message = HTML()
        message.value = f"<b>{code}</b><br>{rows[0][4]}"
        markers.append(
            Marker(
                location=(rows[0][0], rows[0][1]),
                draggable=False,
                popup=message,
                icon=AwesomeIcon(name="train", marker_color="green"),
            )
        )
        for row in rows:
            message = HTML()
            message.value = f"{row[5]}<br>{row[6]}m"
            markers.append(
                Marker(
                    location=(row[2], row[3]),
                    draggable=False,
                    popup=message,
                    icon=AwesomeIcon(name="bicycle"),
                )
            )

        map_markers.add(
            MarkerCluster(
                markers=markers,
                spiderfy_on_max_zoom=False,
                disable_clustering_at_zoom=14,
            )
        )


draw_markers({"new": 400})

max_distance_slider.observe(draw_markers, names="value")